In [3]:
#!/usr/bin/env python3

import types
from owlready2 import get_ontology, Thing, ObjectPropertyClass
import rdflib
import os
import io
import warnings

warnings.filterwarnings("ignore")

# Temporary Ontology Link
# ONTOLOGY_PATH = r'https://github.com/gryvity/MIAPPEx/onto_build/lab_ontologies/miappeXsosa.owl'
ONTOLOGY_PATH = r'miappeXsosa/miappeXsosa.owl'

# Import existiing Ontologies
IMPORT_LOCAL = {
    'ppeo' : os.path.join(r'external/ppeo.owl'),
    'sosa' : os.path.join(r'external/sosa.owl'), 
    'ssn'  : os.path.join(r'external/ssn.ttl')
    }


# THESE NEED A REDESIGN AS WELL
ONTOLOGIES_IMPORT = {
    'ppeo' : [],
    'sosa' : [],
    'ssn'  : []
}

CLASSES = ONTOLOGIES_IMPORT.copy()
# DATA_PROPERTIES = ONTOLOGIES_IMPORT.copy()
# OBJECT_PROPERTIES = ONTOLOGIES_IMPORT.copy()

# PROPERTIES = {'domain': None, 'range': None, 'name': None}

# set classes


# FOR NOW WE WILL PICK A FEW CLASSES TO TEST THE FUNCTIONALITY (LATER WE WILL IMPORT EVERYTHING)
CLASSES['ppeo'].extend(
    [
    'investigation',
    'person',
    'study',
    'observation_unit',
    'observed_variable',
    'biological_material',
    'factor',
    'environment'
    ]
)

CLASSES['sosa'].extend(
    [
    'Observation',
    'FeatureOfInterest',
    'Procedure',
    'ObservableProperty',
    'Sensor',
    'Platform'
    ]
)

CLASSES['ssn'].extend(
    [
    'System',
    'Input',
    'Output',
    'Deployment'
    ]
)




class OntologyBuilder(object):
    IMPORT = {
        'ppeo' : ('http://ppeo.org/ontology/ppeo.owl', 'owl'),
        'sosa' : ('http://www.w3.org/ns/sosa/', 'ttl'),
        'ssn'  : ('http://www.w3.org/ns/ssn/', 'ttl')
    }

    def __init__(self, onto_IRI=ONTOLOGY_PATH, onto_path=ONTOLOGY_PATH, refresh=True):

        self.onto_path = onto_path
        self.onto = get_ontology(onto_IRI)

        if refresh:
            self.onto.imported_ontologies.clear()
        self._load_import()

   
    def _load_import(self):

        for name, (ontology_path, frmt) in self.IMPORT.items():
            match frmt:
                case 'owl':
                    self.onto.imported_ontologies.append(get_ontology(ontology_path))
                case 'ttl':
                    g = rdflib.Graph()
                    g.parse(ontology_path, format='turtle')
                    with io.BytesIO(g.serialize(format='pretty-xml').encode('utf-8')) as temp_file:
                        self.onto.imported_ontologies.append(get_ontology(ontology_path).load(fileobj=temp_file))
                case _:
                    raise ValueError(f'Unsupported format: {frmt}')
            
                

       
    
    def save(self, path=None):
        if path is None:
            path = self.onto_path
        else:
            if not os.path.exists(os.path.dirname(path)):
                os.makedirs(os.path.dirname(path))
        self.onto.save(file = path, format = 'rdfxml')

    def unify_name(self, name):
        if name.islower():
            name = name.title()
        if '_' in name:
            return name.replace('_', ' ').title().replace(' ', '')
        else:
            return name

    def _load_classes_test(self):
        for ont in self.onto.imported_ontologies:
            if ont.name in CLASSES:
                for class_name in ont.classes():
                    if class_name.name in CLASSES[ont.name]:
                        self.make_class(self.unify_name(class_name.name), parent_class=class_name)

    def make_class(self, class_name, parent_class=Thing):
        with self.onto:
            NewClass = types.new_class(class_name, (parent_class,))
            NewClass.is_a = [parent_class]
         
        return NewClass
    


# Test

onto = OntologyBuilder()
onto.save()
for o in onto.onto.imported_ontologies:
    print(o.base_iri)



http://ppeo.org/ontology/ppeo.owl#
http://www.w3.org/ns/sosa/
http://www.w3.org/ns/ssn/


In [ ]:
# Trash
# 
"""Here let us collect all available classes and object properties in form as 
        
        [classname] : {
            'superclasses' : [list of superclasses (is_a)],
            'iri' : class_iri,
            'name' : classname,
            'comment' : comment,
            'index' : index in imported ontologies,
            'imported_from' : self.onto.imported_ontologies[index].name

            'object_properties' : {
                ['property_name'] : {
                    'iri' : property_iri,
                    'comment' : comment,
                    'domain' : domain,
                    'range' : range}
                }
            'data_properties' : {
                ['property_name'] : {
                    'iri' : property_iri,
                    'comment' : comment,
                    'domain' : domain,
                    'range' : range}
                }
"""   


print([test.name for test in onto.onto.get_imported_ontologies()])

ns = 'ppeo'
cn = 'investigation'

# Step 1: Identify Index of imported Ontology
inx = [o.name for o in onto.onto.get_imported_ontologies()].index(ns)

picked_cl = [c for c in onto.onto.get_imported_ontologies()[inx].classes() if c.name == cn].pop()

print(picked_cl.name)
for prop in [p for p in picked_cl.get_class_properties() if isinstance(p, ObjectPropertyClass)]:
    print(f' - {prop.name}, domain: {prop.domain}, range: {prop.range}, type: {type(prop)}')

# Based on the code above we redesign the following mehtods

# class Test(OntologyBuilder):
#     def __init__(self, onto_IRI=ONTOLOGY_PATH, onto_path=ONTOLOGY_PATH, import_local=IMPORT_LOCAL, refresh=True):
#         super().__init__(onto_IRI, onto_path, import_local, refresh )
#         self.onto_content = {}

#     def _load_ontology_content(self):
#         for ont 


"""
    def _load_classes(self):
        for ont in self.onto.imported_ontologies:
            if ont.name in CLASSES:
                for class_name in ont.classes():
                    if class_name.name in CLASSES[ont.name]:
                        self.make_class(self.unify_name(class_name.name), parent_class=class_name)

    def make_class(self, class_name, parent_class=Thing):
        with self.onto:
            NewClass = types.new_class(class_name, (parent_class,))
            NewClass.is_a = [parent_class]
         
        return NewClass
 """


# Strategy

- Make a new ontology which imports a small sett of classes from sosa, ssn and ppeo
- Find a visualization tool (or draw a data model yourself)
- For each class create a json-object which serves as a container of data
- Write a converter from MIAPPE tables to JSON-LD format


# Example of Study

 <!-- http://purl.org/ppeo/PPEO.owl#study -->

    <owl:Class rdf:about="http://purl.org/ppeo/PPEO.owl#study">
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasBiologicalMaterial"/>
                <owl:someValuesFrom rdf:resource="http://purl.org/ppeo/PPEO.owl#biological_material"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasPart"/>
                <owl:someValuesFrom rdf:resource="http://purl.org/ppeo/PPEO.owl#observation_unit"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasPart"/>
                <owl:someValuesFrom rdf:resource="http://purl.org/ppeo/PPEO.owl#observed_variable"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#partOf"/>
                <owl:someValuesFrom rdf:resource="http://purl.org/ppeo/PPEO.owl#investigation"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasContactInstitution"/>
                <owl:qualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:qualifiedCardinality>
                <owl:onClass rdf:resource="http://purl.org/ppeo/PPEO.owl#institution"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasGrowthFacility"/>
                <owl:qualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:qualifiedCardinality>
                <owl:onClass rdf:resource="http://purl.org/ppeo/PPEO.owl#growth_facility"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasLocation"/>
                <owl:qualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:qualifiedCardinality>
                <owl:onClass rdf:resource="http://purl.org/ppeo/PPEO.owl#named_location"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasExperimentalDesign"/>
                <owl:qualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:qualifiedCardinality>
                <owl:onClass rdf:resource="http://purl.org/ppeo/PPEO.owl#experimental_design"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasEnvironment"/>
                <owl:maxQualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:maxQualifiedCardinality>
                <owl:onClass rdf:resource="http://purl.org/ppeo/PPEO.owl#environment"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasObservationLevelHierarchy"/>
                <owl:maxQualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:maxQualifiedCardinality>
                <owl:onClass rdf:resource="http://purl.org/ppeo/PPEO.owl#observation_level_hierarchy"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasName"/>
                <owl:qualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:qualifiedCardinality>
                <owl:onDataRange rdf:resource="http://www.w3.org/2001/XMLSchema#string"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasObservationUnitDescription"/>
                <owl:qualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:qualifiedCardinality>
                <owl:onDataRange rdf:resource="http://www.w3.org/2001/XMLSchema#string"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasStartDateTime"/>
                <owl:qualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:qualifiedCardinality>
                <owl:onDataRange rdf:resource="http://www.w3.org/2001/XMLSchema#dateTime"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasCulturalPractices"/>
                <owl:maxQualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:maxQualifiedCardinality>
                <owl:onDataRange rdf:resource="http://www.w3.org/2001/XMLSchema#string"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasDescription"/>
                <owl:maxQualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:maxQualifiedCardinality>
                <owl:onDataRange rdf:resource="http://www.w3.org/2001/XMLSchema#string"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasEndDateTime"/>
                <owl:maxQualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:maxQualifiedCardinality>
                <owl:onDataRange rdf:resource="http://www.w3.org/2001/XMLSchema#dateTime"/>
            </owl:Restriction>
        </rdfs:subClassOf>
        <rdfs:subClassOf>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasIdentifier"/>
                <owl:maxQualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:maxQualifiedCardinality>
                <owl:onDataRange>
                    <rdfs:Datatype>
                        <owl:unionOf rdf:parseType="Collection">
                            <rdf:Description rdf:about="http://www.w3.org/2001/XMLSchema#anyURI"/>
                            <rdf:Description rdf:about="http://www.w3.org/2001/XMLSchema#string"/>
                        </owl:unionOf>
                    </rdfs:Datatype>
                </owl:onDataRange>
            </owl:Restriction>
        </rdfs:subClassOf>
        <hasExactSynonym xml:lang="en">experiment</hasExactSynonym>
        <rdfs:comment xml:lang="en">A study (or experiment) comprises a series of assays (or measurements) of one or more types, undertaken to answer a particular biological question.
It is recommended that a Study have a description.</rdfs:comment>
        <rdfs:label xml:lang="en">study</rdfs:label>
    </owl:Class>
    <owl:Axiom>
        <owl:annotatedSource rdf:resource="http://purl.org/ppeo/PPEO.owl#study"/>
        <owl:annotatedProperty rdf:resource="http://www.w3.org/2000/01/rdf-schema#subClassOf"/>
        <owl:annotatedTarget>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasName"/>
                <owl:qualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:qualifiedCardinality>
                <owl:onDataRange rdf:resource="http://www.w3.org/2001/XMLSchema#string"/>
            </owl:Restriction>
        </owl:annotatedTarget>
        <hasMIAPPESynonym xml:lang="en">study title</hasMIAPPESynonym>
    </owl:Axiom>
    <owl:Axiom>
        <owl:annotatedSource rdf:resource="http://purl.org/ppeo/PPEO.owl#study"/>
        <owl:annotatedProperty rdf:resource="http://www.w3.org/2000/01/rdf-schema#subClassOf"/>
        <owl:annotatedTarget>
            <owl:Restriction>
                <owl:onProperty rdf:resource="http://purl.org/ppeo/PPEO.owl#hasIdentifier"/>
                <owl:maxQualifiedCardinality rdf:datatype="http://www.w3.org/2001/XMLSchema#nonNegativeInteger">1</owl:maxQualifiedCardinality>
                <owl:onDataRange>
                    <rdfs:Datatype>
                        <owl:unionOf rdf:parseType="Collection">
                            <rdf:Description rdf:about="http://www.w3.org/2001/XMLSchema#anyURI"/>
                            <rdf:Description rdf:about="http://www.w3.org/2001/XMLSchema#string"/>
                        </owl:unionOf>
                    </rdfs:Datatype>
                </owl:onDataRange>
            </owl:Restriction>
        </owl:annotatedTarget>
        <hasMIAPPESynonym xml:lang="en">study unique ID</hasMIAPPESynonym>
    </owl:Axiom>


    # Converted into JSON-LD

```json
{
    "@context": {
        "ppeo": "http://purl.org/ppeo/PPEO.owl#",
        "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
        "xsd": "http://www.w3.org/2001/XMLSchema#"
    },
    "@type": "ppeo:study",
    "ppeo:hasName": {
        "@type": "xsd:string",
        "@value": ""
    },
    "ppeo:hasIdentifier": {
        "@type": "xsd:string",
        "@value": ""
    },
    "ppeo:hasBiologicalMaterial": [
        {   "@type": "ppeo:biological_material",
            "@graph": [

        ]}
    ],
    "ppeo:hasPart": [
        {   "@type": "ppeo:observation_unit",
            "@graph": [

        ]},
        {   "@type": "ppeo:observed_variable",
            "@graph": [

        ]}
    ],

In [ ]:
#!/usr/bin/env python3
"""
Simplified converter for the specific OWL structure provided.
"""

import xml.etree.ElementTree as ET
import json
from typing import Dict, List, Any

def parse_study_class(owl_content: str) -> Dict[str, Any]:
    """Parse the study class from OWL content and create JSON-LD template."""
    # Parse XML
    root = ET.fromstring(owl_content)
    
    # Namespaces
    ns = {
        'owl': 'http://www.w3.org/2002/07/owl#',
        'rdf': 'http://www.w3.org/1999/02/22-rdf-syntax-ns#',
        'rdfs': 'http://www.w3.org/2000/01/rdf-schema#'
    }
    
    # Base template
    template = {
        "@context": {
            "ppeo": "http://purl.org/ppeo/PPEO.owl#",
            "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
            "xsd": "http://www.w3.org/2001/XMLSchema#"
        },
        "@type": "ppeo:study"
    }
    
    # Find all restrictions
    restrictions = []
    
    # Direct restrictions in the class definition
    for restriction in root.findall('.//owl:Restriction', ns):
        restrictions.append(restriction)
    
    # Also check in axioms
    for axiom in root.findall('.//owl:Axiom', ns):
        restriction = axiom.find('owl:annotatedTarget/owl:Restriction', ns)
        if restriction is not None:
            restrictions.append(restriction)
    
    # Process each restriction
    for restriction in restrictions:
        # Get property
        prop_elem = restriction.find('owl:onProperty', ns)
        if prop_elem is None:
            continue
        
        prop_uri = prop_elem.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource')
        if not prop_uri:
            continue
        
        prop_short = prop_uri.replace('http://purl.org/ppeo/PPEO.owl#', 'ppeo:')
        
        # Check restriction type and create appropriate structure
        # 1. Check for someValuesFrom (object properties)
        some_values = restriction.find('owl:someValuesFrom', ns)
        if some_values is not None:
            target_uri = some_values.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource')
            if target_uri:
                target_short = target_uri.replace('http://purl.org/ppeo/PPEO.owl#', 'ppeo:')
                template[prop_short] = {
                    "@type": target_short,
                    "@graph": []
                }
            continue
        
        # 2. Check for qualifiedCardinality with onClass (object properties with cardinality 1)
        qualified_card = restriction.find('owl:qualifiedCardinality', ns)
        if qualified_card is not None:
            on_class = restriction.find('owl:onClass', ns)
            if on_class is not None:
                target_uri = on_class.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource')
                if target_uri:
                    target_short = target_uri.replace('http://purl.org/ppeo/PPEO.owl#', 'ppeo:')
                    template[prop_short] = {
                        "@type": target_short,
                        "@graph": []
                    }
            else:
                # Check for onDataRange (datatype properties)
                on_datarange = restriction.find('owl:onDataRange', ns)
                if on_datarange is not None:
                    datatype_uri = on_datarange.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource')
                    if datatype_uri:
                        if datatype_uri == 'http://www.w3.org/2001/XMLSchema#string':
                            template[prop_short] = {
                                "@type": "xsd:string",
                                "@value": ""
                            }
                        elif datatype_uri == 'http://www.w3.org/2001/XMLSchema#dateTime':
                            template[prop_short] = {
                                "@type": "xsd:dateTime",
                                "@value": ""
                            }
        
        # 3. Check for maxQualifiedCardinality (optional properties)
        max_card = restriction.find('owl:maxQualifiedCardinality', ns)
        if max_card is not None:
            on_class = restriction.find('owl:onClass', ns)
            on_datarange = restriction.find('owl:onDataRange', ns)
            
            if on_class is not None:
                target_uri = on_class.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource')
                if target_uri:
                    target_short = target_uri.replace('http://purl.org/ppeo/PPEO.owl#', 'ppeo:')
                    template[prop_short] = [{
                        "@type": target_short,
                        "@graph": []
                    }]
            elif on_datarange is not None:
                datatype_uri = on_datarange.get('{http://www.w3.org/1999/02/22-rdf-syntax-ns#}resource')
                if datatype_uri:
                    if datatype_uri == 'http://www.w3.org/2001/XMLSchema#string':
                        template[prop_short] = [{
                            "@type": "xsd:string",
                            "@value": ""
                        }]
                    elif datatype_uri == 'http://www.w3.org/2001/XMLSchema#dateTime':
                        template[prop_short] = [{
                            "@type": "xsd:dateTime",
                            "@value": ""
                        }]
    
    return template

# Example usage
if __name__ == "__main__":
    # Read your OWL file
    with open("example.owl", "r", encoding="utf-8") as f:
        owl_content = f.read()
    
    # Convert to JSON-LD template
    template = parse_study_class(owl_content)
    
    # Write to file
    with open("study_template.json", "w", encoding="utf-8") as f:
        json.dump(template, f, indent=2, ensure_ascii=False)
    
    print("Template created successfully!")
    print(json.dumps(template, indent=2))